In [7]:
import openai
from openai import OpenAI
import PyPDF2
import pandas as pd
import json

# Api key
client_openai = OpenAI(api_key = 'API KEY HERE')

# Read the CSV
df = pd.read_csv('FILE PATH OF CSV HERE')

In [8]:
def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(pdf_file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text

# for production change to input
pdf_filename = "PDF FILE HERE"
order_text = extract_text_from_pdf(pdf_filename)

In [9]:
response = client_openai.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": (
                "1: Your task is to convert an incoming text string into a clear Python dictionary within a list."
                "2: search for the following features while doing so:"
                "product_name,article_number,quantity,unit_of_measure,unit_price,expected_arrival"
                "3: you must return the values in that order"
                "4: then add them to a dictionary with the name as the key, and the feature you found for the key as a value."
                "5: VERY IMPORTANT, do not say anything else in your response, you must limit yourself to just sending the dictionaries within a list"
                "6: VERY IMPORTANT the output should be a list with JSON formatting, that doesnt mean you should add ```json at the beginning and end"
                "7: If you see a dot inbetween a number for quantity, example: 1.000.000, then that means 1,000,000 quantity. change it to be a comma instead"
                "8: when taking the unit price, make sure to add the euro symbol upfront for example: €10,00"
            )
        },

        {"role": "user", "content": f"The following is the text you need to search for these values:\n{order_text}"}
    ]
)

response_content = response.choices[0].message.content

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [4]:
new_rows = json.loads(response_content)
new_rows_df = pd.DataFrame(new_rows)
df = pd.concat([df, new_rows_df], ignore_index=True)
df.to_csv('FILE PATH OF CSV HERE', index=False)

<class 'list'>
